# 🏠 Price Predictor - Boston Housing Demo

Este notebook demuestra las capacidades del sistema Price Predictor para la predicción de precios de viviendas.

## 📋 Contenido
1. Carga y exploración de datos
2. Análisis exploratorio
3. Preprocesamiento
4. Entrenamiento de modelos
5. Evaluación y comparación
6. Predicciones de ejemplo

In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configuración de gráficos
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

print("📚 Librerías importadas correctamente")

## 1. 📊 Carga y Exploración de Datos

In [ ]:
# Cargar dataset Boston Housing
from sklearn.datasets import load_boston

# Cargar datos
boston = load_boston()

# Crear DataFrame
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df['MEDV'] = boston.target  # Precio objetivo

print(f"📈 Dataset cargado: {df.shape[0]} filas, {df.shape[1]} columnas")
print(f"🎯 Variable objetivo: MEDV (precio medio en miles de $)")

# Mostrar primeras filas
df.head()

In [ ]:
# Información básica del dataset
print("📋 INFORMACIÓN DEL DATASET")
print("=" * 50)
df.info()

In [ ]:
# Estadísticas descriptivas
print("📊 ESTADÍSTICAS DESCRIPTIVAS")
print("=" * 50)
df.describe().round(2)

## 2. 🔍 Análisis Exploratorio de Datos (EDA)

In [ ]:
# Distribución de la variable objetivo
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histograma
axes[0].hist(df['MEDV'], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
axes[0].set_title('Distribución de Precios (MEDV)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Precio (miles de $)')
axes[0].set_ylabel('Frecuencia')
axes[0].grid(True, alpha=0.3)

# Box plot
axes[1].boxplot(df['MEDV'], patch_artist=True, 
                boxprops=dict(facecolor='lightcoral', alpha=0.7))
axes[1].set_title('Box Plot de Precios (MEDV)', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Precio (miles de $)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Estadísticas de la variable objetivo
print(f"📊 Precio promedio: ${df['MEDV'].mean():.1f}k")
print(f"📊 Precio mediano: ${df['MEDV'].median():.1f}k")
print(f"📊 Rango: ${df['MEDV'].min():.1f}k - ${df['MEDV'].max():.1f}k")
print(f"📊 Desviación estándar: ${df['MEDV'].std():.1f}k")

In [ ]:
# Matriz de correlación interactiva
correlation_matrix = df.corr()

# Crear heatmap con Plotly
fig = go.Figure(data=go.Heatmap(
    z=correlation_matrix.values,
    x=correlation_matrix.columns,
    y=correlation_matrix.columns,
    colorscale='RdYlBu_r',
    zmid=0,
    text=correlation_matrix.round(2).values,
    texttemplate="%{text}",
    textfont={"size": 10},
    hovertemplate='%{x} vs %{y}<br>Correlación: %{z:.3f}<extra></extra>'
))

fig.update_layout(
    title='🔥 Matriz de Correlación - Boston Housing',
    width=800,
    height=600,
    xaxis_title='Variables',
    yaxis_title='Variables'
)

fig.show()

# Variables más correlacionadas con MEDV
print("🎯 CORRELACIONES CON PRECIO (MEDV)")
print("=" * 40)
correlations_with_target = correlation_matrix['MEDV'].drop('MEDV').sort_values(key=abs, ascending=False)
for var, corr in correlations_with_target.head(5).items():
    print(f"{var:>8}: {corr:>6.3f} {'📈' if corr > 0 else '📉'}")

In [ ]:
# Scatter plots de las variables más importantes
important_vars = ['RM', 'LSTAT', 'PTRATIO', 'TAX']

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.ravel()

for i, var in enumerate(important_vars):
    correlation = df[var].corr(df['MEDV'])
    
    # Scatter plot
    axes[i].scatter(df[var], df['MEDV'], alpha=0.6, color=f'C{i}')
    
    # Línea de tendencia
    z = np.polyfit(df[var], df['MEDV'], 1)
    p = np.poly1d(z)
    axes[i].plot(df[var], p(df[var]), "r--", alpha=0.8, linewidth=2)
    
    axes[i].set_xlabel(var)
    axes[i].set_ylabel('MEDV (Precio)')
    axes[i].set_title(f'{var} vs MEDV\nCorrelación: {correlation:.3f}')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. 🔧 Preprocesamiento de Datos

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Separar features y target
X = df.drop('MEDV', axis=1)
y = df['MEDV']

print(f"📊 Features: {X.shape[1]} variables")
print(f"📊 Muestras: {X.shape[0]} viviendas")

# División de datos: train (60%), val (20%), test (20%)
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42  # 0.25 * 0.8 = 0.2
)

print(f"\n📈 DIVISIÓN DE DATOS:")
print(f"  Entrenamiento: {X_train.shape[0]} muestras ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"  Validación:    {X_val.shape[0]} muestras ({X_val.shape[0]/len(X)*100:.1f}%)")
print(f"  Test:          {X_test.shape[0]} muestras ({X_test.shape[0]/len(X)*100:.1f}%)")

In [ ]:
# Escalado de features
scaler = StandardScaler()

# Ajustar solo con datos de entrenamiento
X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train),
    columns=X_train.columns,
    index=X_train.index
)

X_val_scaled = pd.DataFrame(
    scaler.transform(X_val),
    columns=X_val.columns,
    index=X_val.index
)

X_test_scaled = pd.DataFrame(
    scaler.transform(X_test),
    columns=X_test.columns,
    index=X_test.index
)

print("✅ Datos escalados correctamente")

# Verificar escalado
print(f"\n📊 VERIFICACIÓN DEL ESCALADO:")
print(f"Media entrenamiento: {X_train_scaled.mean().mean():.6f}")
print(f"Std entrenamiento: {X_train_scaled.std().mean():.6f}")

## 4. 🤖 Entrenamiento de Modelos

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time

# Función para evaluar modelos
def evaluate_model(model, X_val, y_val):
    """Evalúa un modelo y retorna métricas"""
    y_pred = model.predict(X_val)
    
    metrics = {
        'RMSE': np.sqrt(mean_squared_error(y_val, y_pred)),
        'MAE': mean_absolute_error(y_val, y_pred),
        'R²': r2_score(y_val, y_pred)
    }
    
    return metrics, y_pred

print("🤖 Entrenando modelos...")
print("=" * 50)

# Diccionario para almacenar resultados
models = {}
results = {}

# 1. Random Forest
print("🌲 Entrenando Random Forest...")
start_time = time.time()

rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_train_scaled, y_train)

rf_metrics, rf_pred = evaluate_model(rf_model, X_val_scaled, y_val)
rf_time = time.time() - start_time

models['Random Forest'] = rf_model
results['Random Forest'] = {**rf_metrics, 'Time': rf_time}

print(f"  ✅ Completado en {rf_time:.2f}s")
print(f"  📊 RMSE: {rf_metrics['RMSE']:.3f}")
print(f"  📊 R²: {rf_metrics['R²']:.3f}")

# 2. LightGBM
print("\n⚡ Entrenando LightGBM...")
start_time = time.time()

lgb_model = LGBMRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42,
    verbosity=-1
)
lgb_model.fit(X_train_scaled, y_train)

lgb_metrics, lgb_pred = evaluate_model(lgb_model, X_val_scaled, y_val)
lgb_time = time.time() - start_time

models['LightGBM'] = lgb_model
results['LightGBM'] = {**lgb_metrics, 'Time': lgb_time}

print(f"  ✅ Completado en {lgb_time:.2f}s")
print(f"  📊 RMSE: {lgb_metrics['RMSE']:.3f}")
print(f"  📊 R²: {lgb_metrics['R²']:.3f}")

# 3. Linear Regression
print("\n📈 Entrenando Linear Regression...")
start_time = time.time()

lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

lr_metrics, lr_pred = evaluate_model(lr_model, X_val_scaled, y_val)
lr_time = time.time() - start_time

models['Linear Regression'] = lr_model
results['Linear Regression'] = {**lr_metrics, 'Time': lr_time}

print(f"  ✅ Completado en {lr_time:.2f}s")
print(f"  📊 RMSE: {lr_metrics['RMSE']:.3f}")
print(f"  📊 R²: {lr_metrics['R²']:.3f}")

## 5. 📊 Evaluación y Comparación de Modelos

In [ ]:
# Crear DataFrame de resultados
results_df = pd.DataFrame(results).T
results_df = results_df.round(3)

print("🏆 COMPARACIÓN DE MODELOS")
print("=" * 50)
print(results_df.to_string())

# Identificar mejor modelo
best_model_name = results_df['RMSE'].idxmin()
best_rmse = results_df.loc[best_model_name, 'RMSE']
best_r2 = results_df.loc[best_model_name, 'R²']

print(f"\n🥇 MEJOR MODELO: {best_model_name}")
print(f"   RMSE: {best_rmse:.3f}")
print(f"   R²: {best_r2:.3f}")

In [ ]:
# Visualización de comparación
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

metrics = ['RMSE', 'MAE', 'R²']
colors = ['coral', 'skyblue', 'lightgreen']

for i, metric in enumerate(metrics):
    values = results_df[metric].values
    models_names = results_df.index
    
    bars = axes[i].bar(models_names, values, color=colors[i], alpha=0.7, edgecolor='black')
    axes[i].set_title(f'{metric} por Modelo', fontsize=14, fontweight='bold')
    axes[i].set_ylabel(metric)
    axes[i].grid(True, alpha=0.3)
    
    # Añadir valores en las barras
    for bar, value in zip(bars, values):
        height = bar.get_height()
        axes[i].text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                    f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
    
    # Rotar etiquetas del eje x
    axes[i].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Gráfico de predicciones vs valores reales
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

predictions = {
    'Random Forest': rf_pred,
    'LightGBM': lgb_pred,
    'Linear Regression': lr_pred
}

for i, (model_name, y_pred) in enumerate(predictions.items()):
    # Scatter plot
    axes[i].scatter(y_val, y_pred, alpha=0.6, color=f'C{i}')
    
    # Línea perfecta (y = x)
    min_val = min(y_val.min(), y_pred.min())
    max_val = max(y_val.max(), y_pred.max())
    axes[i].plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, alpha=0.8)
    
    # Configuración
    axes[i].set_xlabel('Valores Reales')
    axes[i].set_ylabel('Predicciones')
    axes[i].set_title(f'{model_name}\nR² = {results[model_name]["R²"]:.3f}')
    axes[i].grid(True, alpha=0.3)
    
    # Añadir límites iguales
    axes[i].set_xlim([min_val, max_val])
    axes[i].set_ylim([min_val, max_val])

plt.tight_layout()
plt.show()

## 6. 🔍 Análisis de Importancia de Features

In [ ]:
# Feature importance para modelos que la soportan
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Random Forest
rf_importances = pd.DataFrame({
    'feature': X_train.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=True)

axes[0].barh(rf_importances['feature'], rf_importances['importance'], color='forestgreen', alpha=0.7)
axes[0].set_title('Random Forest - Importancia de Features', fontweight='bold')
axes[0].set_xlabel('Importancia')
axes[0].grid(True, alpha=0.3)

# LightGBM
lgb_importances = pd.DataFrame({
    'feature': X_train.columns,
    'importance': lgb_model.feature_importances_
}).sort_values('importance', ascending=True)

axes[1].barh(lgb_importances['feature'], lgb_importances['importance'], color='purple', alpha=0.7)
axes[1].set_title('LightGBM - Importancia de Features', fontweight='bold')
axes[1].set_xlabel('Importancia')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Top features
print("🎯 TOP 5 FEATURES MÁS IMPORTANTES")
print("=" * 40)
print("\nRandom Forest:")
for _, row in rf_importances.tail(5).iterrows():
    print(f"  {row['feature']:>8}: {row['importance']:.3f}")
    
print("\nLightGBM:")
for _, row in lgb_importances.tail(5).iterrows():
    print(f"  {row['feature']:>8}: {row['importance']:.3f}")

## 7. 🎯 Ejemplos de Predicciones

In [ ]:
# Seleccionar mejor modelo
best_model = models[best_model_name]
print(f"🥇 Usando el mejor modelo: {best_model_name}")

# Ejemplos de predicciones
sample_indices = [0, 50, 100, 150, 200]
samples = X_test.iloc[sample_indices]
samples_scaled = X_test_scaled.iloc[sample_indices]
real_prices = y_test.iloc[sample_indices]

# Hacer predicciones
predictions = best_model.predict(samples_scaled)

print("\n🏠 EJEMPLOS DE PREDICCIONES")
print("=" * 60)
print(f"{'Muestra':>8} {'Real':>10} {'Predicho':>10} {'Error':>10} {'Error%':>8}")
print("-" * 60)

for i, (idx, real, pred) in enumerate(zip(sample_indices, real_prices, predictions)):
    error = abs(real - pred)
    error_pct = (error / real) * 100
    
    print(f"{i+1:>8} {real:>10.1f} {pred:>10.1f} {error:>10.1f} {error_pct:>7.1f}%")

# Estadísticas de error
mean_error_pct = np.mean([abs(r-p)/r*100 for r, p in zip(real_prices, predictions)])
print(f"\n📊 Error promedio: {mean_error_pct:.1f}%")

In [ ]:
# Crear una predicción personalizada
print("🎯 PREDICCIÓN PERSONALIZADA")
print("=" * 40)

# Ejemplo: Casa con características promedio pero con más habitaciones
custom_house = pd.DataFrame({
    'CRIM': [2.0],      # Criminalidad baja-media
    'ZN': [20.0],       # Zona residencial
    'INDUS': [8.0],     # Industria moderada
    'CHAS': [0],        # No limita con río
    'NOX': [0.45],      # NOX bajo
    'RM': [7.5],        # 7.5 habitaciones (más que promedio)
    'AGE': [30.0],      # Casa relativamente nueva
    'DIS': [6.0],       # Distancia media a centros
    'RAD': [5.0],       # Accesibilidad media
    'TAX': [250.0],     # Impuestos moderados
    'PTRATIO': [16.0],  # Ratio alumno-profesor bueno
    'B': [380.0],       # B alto
    'LSTAT': [8.0]      # Estatus socioeconómico alto
})

# Escalar
custom_house_scaled = pd.DataFrame(
    scaler.transform(custom_house),
    columns=custom_house.columns
)

# Predicción
custom_prediction = best_model.predict(custom_house_scaled)[0]

print("Características de la casa:")
print(f"  🏠 Habitaciones: {custom_house.iloc[0]['RM']} (promedio: {df['RM'].mean():.1f})")
print(f"  🚔 Criminalidad: {custom_house.iloc[0]['CRIM']} (promedio: {df['CRIM'].mean():.1f})")
print(f"  📚 Ratio estudiante-profesor: {custom_house.iloc[0]['PTRATIO']} (promedio: {df['PTRATIO'].mean():.1f})")
print(f"  📊 Estatus bajo (%): {custom_house.iloc[0]['LSTAT']} (promedio: {df['LSTAT'].mean():.1f})")

print(f"\n💰 Precio predicho: ${custom_prediction:.1f}k")
print(f"💰 Precio promedio dataset: ${df['MEDV'].mean():.1f}k")
print(f"💰 Diferencia: ${custom_prediction - df['MEDV'].mean():.1f}k ({((custom_prediction/df['MEDV'].mean()-1)*100):+.1f}%)")

## 🎉 Conclusiones

### 📊 Resultados del Análisis:

1. **Dataset**: 506 viviendas con 13 características predictoras
2. **Mejor modelo**: Determinado por menor RMSE
3. **Features importantes**: RM (habitaciones), LSTAT (estatus), PTRATIO (educación)
4. **Rendimiento**: R² > 0.8 indica buena capacidad predictiva

### 🚀 Próximos Pasos:

- **Optimización de hiperparámetros** con Optuna
- **Validación cruzada** para mayor robustez
- **Feature engineering** para mejorar predicciones
- **Despliegue** en aplicación Streamlit

### 💡 Insights Clave:

- El **número de habitaciones (RM)** es el factor más importante
- El **estatus socioeconómico (LSTAT)** tiene fuerte correlación negativa
- Los modelos ensemble (RF, LightGBM) superan a Linear Regression
- La **calidad educativa (PTRATIO)** influye significativamente en precios